In [1]:
# /Users/teliov/TUD/Thesis/Medvice/Notebooks/data/04_06_new_data/data/split
# So we can use the *thesislib* package
import sys
import os

module_path = os.path.abspath("..")

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from thesislib.utils import pathutils
import json

In [3]:
from thesislib.utils.ml import process
from thesislib.utils.ml import runners, models

In [25]:
import importlib
_ = importlib.reload(process)
_ = importlib.reload(runners)

In [5]:
data_dir = pathutils.get_data_file("05_27_nlice")
nlice_data_dir = os.path.join(data_dir, "nlice")

In [6]:
nlice_module_dir = "/Users/teliov/TUD/symcat-to-synthea/output/module_ai_med_adv"
nlice_data_csv = pathutils.get_data_file("05_27_nlice/ai/output_med_ai_adv/symptoms/csv/symptoms.csv")

In [7]:
nlice_op_data_dir = os.path.join(nlice_data_dir, "data")
# split into a train and test set
nlice_train_file, nlice_test_file = process.split_data(nlice_data_csv, nlice_op_data_dir)

nlice_parsed_data_dir = os.path.join(nlice_op_data_dir, "parsed")

In [8]:
transformation_map_file = os.path.join(nlice_op_data_dir, "transformation_map.json")
encoding_map_file = os.path.join(nlice_op_data_dir, "encoding_map.json")
reduction_map_file = os.path.join(nlice_op_data_dir, "reduction_map.json")
with open(transformation_map_file) as fp:
    transformation_map = json.load(fp)
with open(encoding_map_file) as fp:
    encoding_map = json.load(fp)
with open(reduction_map_file) as fp:
    reduction_map = json.load(fp)

In [9]:
nlice_parsed_data_dir = os.path.join(nlice_op_data_dir, "parsed")

In [10]:
data_dir = pathutils.get_data_file("05_27_nlice")
basic_data_dir = os.path.join(data_dir, "basic")
basic_symptom_map_file = os.path.join(basic_data_dir, "symptom_db.json")
basic_condition_map_file = os.path.join(basic_data_dir, "condition_db.json")

In [11]:
# parse the train set and let's train
nlice_parsed_train = process.parse_data(
    nlice_train_file,
    basic_condition_map_file,
    basic_symptom_map_file,
    nlice_parsed_data_dir,
    is_nlice=True,
    transform_map=transformation_map,
    encode_map=encoding_map,
    reduce_map=reduction_map
)

In [12]:
# train with RF and then with NB
nlice_rf_dir = os.path.join(nlice_op_data_dir, "output/rf")
rfparams = models.RFParams()
rfparams.n_estimators = 200
rfparams.max_depth = None

run_ok = runners.train_rf(
    nlice_parsed_train,
    basic_symptom_map_file,
    nlice_rf_dir,
    rfparams,
    "NLICE AI-MED Run",
    "local-pc",
    True
)

Calculating Accuracy


In [ ]:
# train NB
nlice_nb_dir = os.path.join(nlice_op_data_dir, "output/nb")

run_ok = runners.train_nb(
    nlice_parsed_train,
    basic_symptom_map_file,
    nlice_nb_dir,
    "NLICE AI-MED Run",
    "local-pc",
    True
)

In [20]:
symptom_map = {code: str(idx) for idx, code in enumerate(sorted(symptoms_db.keys()))}

In [27]:
import numpy as np

In [34]:
l = np.random.random((3,24))

In [35]:
l

array([[0.90971425, 0.11705411, 0.24807595, 0.07558665, 0.82650648,
        0.90207644, 0.12772574, 0.01083891, 0.93498243, 0.1228663 ,
        0.05039713, 0.97078692, 0.2163535 , 0.15006288, 0.77849612,
        0.49769951, 0.48839921, 0.70333114, 0.25166317, 0.31034356,
        0.12309439, 0.47839935, 0.16498618, 0.0505228 ],
       [0.52120171, 0.43037764, 0.12952776, 0.83406575, 0.16547058,
        0.67513176, 0.38057206, 0.05419291, 0.30786714, 0.55994043,
        0.4658807 , 0.86624539, 0.8693866 , 0.15866228, 0.91295045,
        0.27941083, 0.81387083, 0.27554915, 0.60098909, 0.99714982,
        0.81224112, 0.56446684, 0.2902226 , 0.52521559],
       [0.4301576 , 0.87727713, 0.58110042, 0.35842363, 0.08553062,
        0.09581492, 0.94847359, 0.69092392, 0.72450594, 0.18222125,
        0.31223814, 0.45860536, 0.08014731, 0.88476729, 0.08332489,
        0.63228769, 0.09562253, 0.84197379, 0.38852387, 0.69728393,
        0.01258712, 0.79884579, 0.12884759, 0.80639949]])

In [36]:
l[:, 3:4]

array([[0.07558665],
       [0.83406575],
       [0.35842363]])

In [38]:
l[:, 23:]

array([[0.0505228 ],
       [0.52521559],
       [0.80639949]])